In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd /content/drive/MyDrive/Colab Notebooks/pygaggle-master/

/content/drive/MyDrive/Colab Notebooks/pygaggle-master


In [4]:
ls

data/            indexes/     models/     requirements.txt  setup.py
docs/            LICENSE      notebooks/  runs/             tests/
environment.yml  logs/        pygaggle/   scripts/          tools/
experiments/     MANIFEST.IN  README.md   setup.cfg


In [5]:
pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Ignoring dataclasses: markers 'python_version < "3.7"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 121.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 107.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.7/508.7 kB 52.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 92.0 MB/s eta 0:00:00
     ━━━━

In [7]:
cd /content/drive/MyDrive/Colab Notebooks/pygaggle-master/

/content/drive/MyDrive/Colab Notebooks/pygaggle-master


test top 100을 Bert로 rerank하고 재정렬해서 JSON으로 뽑아내는 코드

In [8]:
import json

queryy=[]
#test set의 경우에는 list 사이즈 [3610][100][3]

file_path ='/content/drive/MyDrive/Colab Notebooks/NQ_test_top100_all.json'    #test set 이용
#file_path ='/content/drive/MyDrive/Colab Notebooks/NQ_dev_top100_all.json'    #dev set 이용

with open(file_path, 'r', encoding='utf-8') as file:
      data = json.load(file)
      data_size=len(data)
      corpus = [[['a' for _ in range(5)] for _ in range(100)] for _ in range(data_size)]
      for i in range(len(data)):
       queryy.append(data[i]["question"])
       for j in range(100):
        corpus[i][j][0]=data[i]["ctxs"][j]["id"]
        corpus[i][j][1]=data[i]["ctxs"][j]["title"]
        corpus[i][j][2]=data[i]["ctxs"][j]["text"]
        corpus[i][j][3]=data[i]["ctxs"][j]["score"]
        corpus[i][j][4]=data[i]["ctxs"][j]["has_answer"]

#print(data_size)
#print(type(corpus[0][99][2]))
#print(corpus[0][99][1])
#print(queryy[1])
#print(data[0]["ctxs"][2]["score"])

In [9]:
#Bert 모델 이용할때 이거 실행하면됨
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.transformer import MonoBERT

reranker =  MonoBERT()

2023-05-20 12:44:09 [INFO] utils: NumExpr defaulting to 4 threads.
2023-05-20 12:44:11 [INFO] loader: Loading faiss with AVX2 support.
2023-05-20 12:44:11 [INFO] loader: Successfully loaded faiss with AVX2 support.


In [ ]:
tot_size=0 #100개의 passage중 has_answer가 하나라도 true 질문의 개수
accurate=0 #100개의 passage중 has_answer가 true인 query에 대해 reranked된 top-k에 has_answer가 하나라도 true인 개수

for q in range(data_size):
  print(queryy[q])
  query = Query(queryy[q])
  passages=corpus[q]

  texts = [ Text(p[2], {'id': p[0],'title':p[1],'text':p[2],'score':p[3],'has_answer' : p[4]}, 0) for p in passages] # Note, pyserini scores don't matter since T5 will ignore them.

 # Either option, let's print out the passages prior to reranking:
# for i in range(0, 10):
#     print(f'{i+1:2} {texts[i].metadata["docid"]:15} {texts[i].score:.5f} {texts[i].metadata["has_answer"]:10} {texts[i].text}')

 #기존 passage의 has_answer가 모두 false인지 아닌지 보는것(accuracy계산에 활용), flag=1이면 has_answer가 하나라도 있는거임
  flag=0
  for a in range(100):
    if texts[a].metadata["has_answer"]==1:
      flag=1
      tot_size+=1
      break

  #print(flag)
 # Finally, rerank:
  reranked = reranker.rerank(query, texts)

  count=0

 # Print out reranked results:
  for i in range(0, 10):
      print(f'{i+1:2} {reranked[i].metadata["id"]:15} {reranked[i].score:.5f}  {reranked[i].metadata["has_answer"]:6} {reranked[i].text} {reranked[i].metadata["title"]:20}') #세번째가 0으로 출력되면 has answer: False, 1로 출력되면 has answer: True
      
      if flag==1:
       if reranked[i].metadata["has_answer"]==1:
        count+=1

  if count>0:
   accurate+=1

  #json 파일 값 변경
  for k in range(100):
    data[q]["ctxs"][k]["id"]=reranked[k].metadata["id"]
    data[q]["ctxs"][k]["title"]=reranked[k].metadata["title"]
    data[q]["ctxs"][k]["text"]=reranked[k].metadata["text"]
    data[q]["ctxs"][k]["score"]=str(reranked[k].score)  #원래 파일 형식에서 score가 ""형식이라 이렇게함
    data[q]["ctxs"][k]["has_answer"]=reranked[k].metadata["has_answer"]

  accuracy=accurate/tot_size
  print(f'Accuracy : {accuracy:.4f}')

with open('modified.json', 'w') as file:
      json.dump(data, file)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 5 wiki:11798610   -0.00586       0 layout to the Gavilan, which used the Intel 80186 processor. The year 1983 also saw the launch of what was probably the biggest-selling early laptop, the Kyocera Kyotronic 85. Owing much to the design of the previous Epson HX-20, and although at first a slow seller in Japan, it was quickly licensed by Tandy Corporation, Olivetti, and NEC, who recognised its potential and marketed it respectively as the TRS-80 Model 100 line (or Tandy 100), Olivetti M-10, and NEC PC-8201. The machines ran on standard AA batteries. The Tandy's built-in programs, including a BASIC interpreter, a text editor, and a History of laptops  
 6 wiki:1618926    -0.00594       1 was the Epson HX-20, invented (patented) by Suwa Seikosha's Yukio Yokozawa in July 1980, introduced at the COMDEX computer show in Las Vegas by Japanese company Seiko Epson in 1981, and widely released in 1982. It had an LCD screen, a rechargeable battery, and a calcul

test top 100을 T5로 rerank하고 재정렬해서 JSON으로 뽑아내는 코드

In [ ]:
import json

queryy=[]
#test set의 경우에는 list 사이즈 [3610][100][3]

file_path ='/content/drive/MyDrive/Colab Notebooks/NQ_test_top100_all.json'    #test set 이용할땐 이거 주석 지우기
#file_path ='/content/drive/MyDrive/Colab Notebooks/NQ_dev_top100_all.json'    #dev set 이용할땐 이거 주석 지우기

with open(file_path, 'r', encoding='utf-8') as file:
      data = json.load(file)
      data_size=len(data)
      corpus = [[['a' for _ in range(5)] for _ in range(100)] for _ in range(data_size)]
      for i in range(len(data)):
       queryy.append(data[i]["question"])
       for j in range(100):
        corpus[i][j][0]=data[i]["ctxs"][j]["id"]
        corpus[i][j][1]=data[i]["ctxs"][j]["title"]
        corpus[i][j][2]=data[i]["ctxs"][j]["text"]
        corpus[i][j][3]=data[i]["ctxs"][j]["score"]
        corpus[i][j][4]=data[i]["ctxs"][j]["has_answer"]

#print(data_size)
#print(type(corpus[0][99][2]))
#print(corpus[0][99][1])
#print(queryy[1])
#print(data[0]["ctxs"][2]["score"])

In [ ]:
#T5 모델 이용
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.transformer import MonoT5

reranker =  MonoT5()

In [ ]:
tot_size=0 #100개의 passage중 has_answer가 하나라도 true 질문의 개수
accurate=0 #100개의 passage중 has_answer가 true인 query에 대해 reranked된 top-k에 has_answer가 하나라도 true인 개수

for q in range(data_size):
  print(queryy[q])
  query = Query(queryy[q])
  passages=corpus[q]

  texts = [ Text(p[2], {'id': p[0],'title':p[1],'text':p[2],'score':p[3],'has_answer' : p[4]}, 0) for p in passages] # Note, pyserini scores don't matter since T5 will ignore them.

 # Either option, let's print out the passages prior to reranking:
# for i in range(0, 10):
#     print(f'{i+1:2} {texts[i].metadata["docid"]:15} {texts[i].score:.5f} {texts[i].metadata["has_answer"]:10} {texts[i].text}')

 #기존 passage의 has_answer가 모두 false인지 아닌지 보는것(accuracy계산에 활용), flag=1이면 has_answer가 하나라도 있는거임
  flag=0
  for a in range(100):
    if texts[a].metadata["has_answer"]==1:
      flag=1
      tot_size+=1
      break

  #print(flag)
 # Finally, rerank:
  reranked = reranker.rerank(query, texts)

  count=0

 # Print out reranked results:
  for i in range(0, 10):
      print(f'{i+1:2} {reranked[i].metadata["id"]:15} {reranked[i].score:.5f}  {reranked[i].metadata["has_answer"]:6} {reranked[i].text} {reranked[i].metadata["title"]:20}') #세번째가 0으로 출력되면 has answer: False, 1로 출력되면 has answer: True
      
      if flag==1:
       if reranked[i].metadata["has_answer"]==1:
        count+=1

  if count>0:
   accurate+=1

  #json 파일 값 변경
  for k in range(100):
    data[q]["ctxs"][k]["id"]=reranked[k].metadata["id"]
    data[q]["ctxs"][k]["title"]=reranked[k].metadata["title"]
    data[q]["ctxs"][k]["text"]=reranked[k].metadata["text"]
    data[q]["ctxs"][k]["score"]=str(reranked[k].score)  #원래 파일 형식에서 score가 ""형식이라 이렇게함
    data[q]["ctxs"][k]["has_answer"]=reranked[k].metadata["has_answer"]

  accuracy=accurate/tot_size
  print(f'Accuracy : {accuracy:.4f}')

with open('nq_test_top100_t5.json', 'w') as file:
      json.dump(data, file)